In [1]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.7
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [2]:
attach(readRDS("sim.Rds"))

In [3]:
M <- M[1:10]           # mean
COV <- COV[1:10,1:10]  # covariance
NCENS <- NCENS[11:20]  # num of censored obs.
OUT_D <- runif(5,7,20) # outliers distance

In [4]:
# Random sample
Y <- mvtnorm::rmvnorm(N, M, COV)

# Outliers
OUT <- mvtnorm::rmvnorm(length(OUT_D), rep(0, length(M))) # std mv normal
OUT <- OUT_D * OUT / apply(OUT, 1, function(x) { norm(rbind(x), type = "F") }) # std mv normal with mahalanobis distance OUT_D
OUT <- OUT %*% chol(cov(Y)) # std mv normal with distance OUT_D and cov the same as Y
OUT <- t(t(OUT) + colMeans(Y)) # std mv normal with distance OUT_D and cov the same as Y and mean the same as Y
OUT_I <- sample(1:nrow(Y), nrow(OUT))

#' Step 1. Mix outliers (uncomment 2nd string)
X0_s1 <- Y
# X0_s1[OUT_I,] <- OUT

#' Step 2. Censor
X0_s2 <- X0_s1
for (i in 1:ncol(Y)) {
  if (NCENS[i] == 0) next();
  X0_s2[order(Y[,i])[1:(NCENS[i])],i] <- NA
}

# Random censored sample with outliers
X0 <- X0_s2

In [5]:
max_iter <- 100
iter <- 0
ll_old <- -Inf
ll <- -1e10
tol <- 1e-10

n <- nrow(Y)
k <- ncol(Y)
LOD <- apply(X0, 2, min, na.rm = T)
mu <- apply(X0, 2, mean, na.rm = T)
cov <- diag(1,k)
hist <- list(ll = list(), mu = list(), cov = list())

while ((iter < max_iter) & ((ll - ll_old) >= tol * abs(ll))) {
    iter <- iter + 1
    X <- X0
    R <- array(0, c(k,k,n))
    for (i in 1:n) {
        o <- !is.na(X0[i,])
        if (sum(o) == k) next()
        m <- mu[!o] + cov[!o,o] %*% solve(cov[o,o]) %*% (X0[i,o] - mu[o])
        v <- cov[!o,!o] - cov[!o,o] %*% solve(cov[o,o]) %*% cov[o,!o]
        mm <- tmvtnorm::mtmvnorm(m[,1], v, upper = LOD[!o])
        X[i,!o] <- mm$tmean
        R[!o,!o,i] <- mm$tvar
    }
    mu <- apply(X, 2, mean)
    S <- sapply(1:n, function(i) {
        (X[i,] - mu) %*% t(X[i,] - mu) + R[,,i]
    })
    cov <- matrix(apply(S, 1, mean), k, k)
    cov <- (cov + t(cov)) / 2

    ll_old <- ll
    ll <- sum(sapply(1:n, function(i) {
        x <- X[i,]
        o <- !is.na(X0[i,])
        quadform <- t(x[o] - mu[o]) %*% solve(cov[o,o]) %*% (x[o] - mu[o])
        logDetSig <- if (sum(o) == 1) { log(cov[o,o]) } else { log(det(cov[o,o])) }
        log_Phi <- if (sum(!o) > 0) {
            m <- mu[!o] + cov[!o,o] %*% solve(cov[o,o]) %*% (x[o] - mu[o])
            v <- cov[!o,!o] - cov[!o,o] %*% solve(cov[o,o]) %*% cov[o,!o]

            log(mvtnorm::pmvnorm(lower = -Inf, upper = LOD[!o], mean = m[,1], sigma = v))
        } else {
            0
        }
        - .5 * quadform - .5 * logDetSig + log_Phi - sum(o) * log(2 * pi) / 2
    }))

    print(str_glue("Iter {iter}, {round(ll_old,5)} -> {round(ll, 5)}"))
    flush.console()

    hist$ll[[iter]] <- ll
    hist$mu[[iter]] <- mu
    hist$cov[[iter]] <- cov
}

Iter 1, -1e+10 -> -112.86183
Iter 2, -112.86183 -> -35.36523
Iter 3, -35.36523 -> -18.04968
Iter 4, -18.04968 -> -11.61173
Iter 5, -11.61173 -> -8.66006
Iter 6, -8.66006 -> -7.20658
Iter 7, -7.20658 -> -6.47017
Iter 8, -6.47017 -> -6.09413
Iter 9, -6.09413 -> -5.90587
Iter 10, -5.90587 -> -5.81192
Iter 11, -5.81192 -> -5.76459
Iter 12, -5.76459 -> -5.74177
Iter 13, -5.74177 -> -5.73113
Iter 14, -5.73113 -> -5.72508
Iter 15, -5.72508 -> -5.72161
Iter 16, -5.72161 -> -5.722


In [6]:
cat("EM: ", sum((M - mu)^2))
cat("\n")
cat("Raw:", sum((M - apply(X0, 2, mean, na.rm = T))^2))

EM:  0.01712496
Raw: 0.1731536

In [7]:
cat("EM: ", sum((COV - cov)^2))
cat("\n")
cat("Raw:", sum((COV - cov(X0, use = "pairwise"))^2))

EM:  0.03204334
Raw: 0.08705788